In [34]:
#pip install tensorflow==2.10

In [35]:
!pip3 install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import tensorflow_hub as thub
import tensorflow_text

In [37]:
import base64
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

import pandas as pd
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')



# Get data

### imgs

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
/content/drive/MyDrive/IMDB

NameError: ignored

In [ ]:
PATH = '/content/drive/MyDrive/lab_2/IMDB_new/final_storage/train'

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)
seed = 42


In [ ]:
train_dir

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/IMDB/final_storage/train',
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            validation_split = 0.2,
                                                            subset = 'training',
                                                            seed = 42)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/IMDB/final_storage/train',
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE,
                                                                 validation_split = 0.2,
                                                                 subset = 'validation',
                                                                 seed = 42)



In [ ]:
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    '/content/drive/MyDrive/IMDB/old_storage_text/train',
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset='training',
    seed=seed)

# class_names = raw_train_ds.class_names
# train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    '/content/drive/MyDrive/IMDB/old_storage_text/train',
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset='validation',
    seed=seed)


#### check sets that coincide 

# Model setup

In [ ]:

class_names = train_dataset.class_names
classes_num  = len(class_names)
print(class_names)

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:

data_augmentation = tf.keras.Sequential([
   tf.keras.layers.RandomFlip('horizontal'),
   tf.keras.layers.RandomRotation(0.2),
])


for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
#IMG_SHAPE = IMG_SIZE 
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(label_batch.shape)

## Model itself

In [ ]:
base_model.trainable = True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = tf.keras.layers.Dense(classes_num)
prediction_batch = prediction_layer(feature_batch_average)


### Prepare data for multi-inputs

In [ ]:

img_features = []
for img_batch, img_label_batch in train_dataset:
    for singe in img_batch:
        img_features.append(singe)
text_features = []
text_labels = []
for text_batch, label_batch in train_ds:
    for single in text_batch:
        text_features.append(single)
    for single in label_batch:
        text_labels.append(single)

In [ ]:
text_features_st = tf.stack(text_features)
img_features_st = tf.stack(img_features)
labels_st = tf.stack(text_labels)

In [ ]:
a = tf.data.Dataset.from_tensor_slices(img_features_st)
b = tf.data.Dataset.from_tensor_slices(text_features_st)
c = tf.data.Dataset.from_tensor_slices(labels_st)

In [ ]:
a = a.batch(32)
b = b.batch(32)
c = c.batch(32)

In [ ]:
combined_dataset = tf.data.Dataset.zip(((a,b),c))

In [ ]:
initial_epochs = 15

history = model.fit(combined_dataset , epochs=initial_epochs)

In [ ]:
img_features = []
for img_batch, img_label_batch in validation_dataset:
    for singe in img_batch:
        img_features.append(singe)
text_features = []
text_labels = []
for text_batch, label_batch in val_ds:
    for single in text_batch:
        text_features.append(single)
    for single in label_batch:
        text_labels.append(single)

In [ ]:
val_text_features_st = tf.stack(text_features)
val_img_features_st = tf.stack(img_features)
val_labels_st = tf.stack(text_labels)

In [ ]:
val_a = tf.data.Dataset.from_tensor_slices(val_img_features_st)
val_b = tf.data.Dataset.from_tensor_slices(val_text_features_st)
val_c = tf.data.Dataset.from_tensor_slices(val_labels_st)

In [ ]:
val_a = val_a.batch(32)
val_b = val_b.batch(32)
val_c = val_c.batch(32)

In [ ]:
validation_combined_dataset = tf.data.Dataset.zip(((val_a,val_b),val_c))

### Try with validation

In [ ]:
initial_epochs = 15
history = model.fit(combined_dataset , epochs=initial_epochs, validation_data=validation_combined_dataset)

### something new and probably special

In [ ]:
base_model.trainable = True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = tf.keras.layers.Dense(classes_num)
prediction_batch = prediction_layer(feature_batch_average)

In [ ]:
try: 
    del model
except:
    2+2

inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs_img = tf.keras.layers.Dense(100)(x)
#outputs_img = prediction_layer(x)

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
# text_input = tf.keras.layers.Input(shape=(),)

# text_input = layers.Embedding(input_dim=max_features+1, output_dim=embedding_dim, input_length=sequence_length, name='vectorized_text')

preprocessing_layer = thub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)

encoder = thub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)

net = outputs['pooled_output']
net = tf.keras.layers.Dense(24, name='classifier')(net)
net = tf.keras.layers.Dropout(0.5)(net)
model_merging = concatenate([outputs_img,net]) 

final = tf.keras.layers.Dense(classes_num, activation='sigmoid', name='finalmente')(model_merging)


model = tf.keras.Model([inputs,text_input], final)

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
import tensorflow.keras.losses as losses
base_learning_rate = 0.0001

# model.compile(
model.compile(
    # loss=losses.BinaryCrossentropy(from_logits=True),
    loss=losses.CategoricalCrossentropy(),
    #optimizer='adam',
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10), 
    # metrics=tf.metrics.BinaryAccuracy(threshold=0.0)
    #metrics=tf.metrics.CategoricalAccuracy()
    metrics=['accuracy']
)

In [ ]:

img_features = []
for img_batch, img_label_batch in train_dataset:
    for singe in img_batch:
        img_features.append(singe)
text_features = []
text_labels = []


In [ ]:
img_features_st = tf.stack(img_features)
#labels_st = tf.stack(text_labels)

In [ ]:
a = tf.data.Dataset.from_tensor_slices(img_features_st)
#b = tf.data.Dataset.from_tensor_slices(text_features_st)
#c = tf.data.Dataset.from_tensor_slices(labels_st)

In [ ]:
a = a.batch(32)
b = b.batch(32)
c = c.batch(32)

In [ ]:
validation_combined_dataset = tf.data.Dataset.zip(((val_a,val_b),val_c))

In [ ]:
img_features = []
for img_batch, img_label_batch in validation_dataset:
    for singe in img_batch:
        img_features.append(singe)
text_features = []
text_labels = []
for text_batch, label_batch in val_ds:
    for single in text_batch:
        text_features.append(single)
    for single in label_batch:
        text_labels.append(single)

# NICE TRY

In [ ]:
train_dataset

In [ ]:
#get images
img_features = []
for img_batch, img_label_batch in train_dataset:
    for singe in img_batch:
        img_features.append(singe)


#get text with images
text_features = []
text_labels = []
for text_batch, label_batch in raw_train_ds:
    for single in text_batch:
        text_features.append(single)
    for single in label_batch:
        text_labels.append(single)






In [ ]:
img_features_st = tf.stack(img_features)
text_features_st = tf.stack(text_features)
labels_st = tf.stack(text_labels)


In [ ]:
img_features_st_np = np.array(img_features_st)
text_features_st_np = np.array(text_features_st)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import json
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#PREPROC TEXT
text_df = pd.DataFrame()
text_df['text'] = text_features_st_np

text_df['text'] = [x.decode("latin-1") for x in text_df['text'].tolist()]


vocab_size = 10000
embedding_dim = 16
max_length = 8250
trunc_type='post'
oov_tok = '<OOV>'
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

tokenizer.fit_on_texts(text_df['text'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(text_df['text'])

padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)



In [ ]:
img_features_st_np.shape
padded.shape

In [ ]:
labels_st_np = np.array(labels_st)

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels_st_np)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[1, :])])

onehot_encoded.shape


In [ ]:
a = tf.data.Dataset.from_tensor_slices(img_features_st_np)
b = tf.data.Dataset.from_tensor_slices(padded)
c = tf.data.Dataset.from_tensor_slices(onehot_encoded)


In [ ]:

a = a.batch(32)
b = b.batch(32)
c = c.batch(32)
combined_dataset = tf.data.Dataset.zip(((a,b),c))

# make validation data 

In [ ]:
#get images
img_features = []
for img_batch, img_label_batch in validation_dataset:
    for singe in img_batch:
        img_features.append(singe)


#get text with images
text_features = []
text_labels = []
for text_batch, label_batch in raw_val_ds:
    for single in text_batch:
        text_features.append(single)
    for single in label_batch:
        text_labels.append(single)


img_features_st = tf.stack(img_features)
text_features_st = tf.stack(text_features)
labels_st = tf.stack(text_labels)

img_features_st_np = np.array(img_features_st)
text_features_st_np = np.array(text_features_st)

#PREPROC TEXT
text_df = pd.DataFrame()
text_df['text'] = text_features_st_np

text_df['text'] = [x.decode("latin-1") for x in text_df['text'].tolist()]


vocab_size = 10000
embedding_dim = 16
max_length = 8250
trunc_type='post'
oov_tok = '<OOV>'
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

tokenizer.fit_on_texts(text_df['text'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(text_df['text'])

padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)


labels_st_np = np.array(labels_st)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels_st_np)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[1, :])])

onehot_encoded.shape


a_val = tf.data.Dataset.from_tensor_slices(img_features_st_np)
b_val = tf.data.Dataset.from_tensor_slices(padded)
c_val = tf.data.Dataset.from_tensor_slices(onehot_encoded)


a_val = a_val.batch(32)
b_val = b_val.batch(32)
c_val = c_val.batch(32)
combined_dataset_val = tf.data.Dataset.zip(((a_val,b_val),c_val))


In [ ]:
from tensorflow.keras.layers import Dense, concatenate
import tensorflow.keras.losses as losses

In [40]:
try: 
    del model
except:
    2+2

inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs_img = tf.keras.layers.Dense(100)(x)
#outputs_img = prediction_layer(x)

text_input = tf.keras.layers.Input(shape=(8250,), name='text')
# text_input = tf.keras.layers.Input(shape=(),)
# text_input = layers.Embedding(input_dim=max_features+1, output_dim=embedding_dim, input_length=sequence_length, name='vectorized_text')

net = tf.keras.layers.Embedding(vocab_size, 24)(text_input)
net = tf.keras.layers.Conv1D(128, 5, activation='relu')(net)
net = tf.keras.layers.GlobalAveragePooling1D()(net)
net = tf.keras.layers.Dense(64, activation='relu')(net)
net = tf.keras.layers.Dense(100)(net)
net = tf.keras.layers.Dropout(0.2)(net)


model_merging = concatenate([outputs_img,net]) 

final = tf.keras.layers.Dense(24, activation='sigmoid', name='finalmente')(model_merging)


model = tf.keras.Model([inputs,text_input], final)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes = True)

In [41]:
import tensorflow.keras.losses as losses
base_learning_rate = 0.0001

# model.compile(
model.compile(
    # loss=losses.BinaryCrossentropy(from_logits=True),
    loss='categorical_crossentropy',
    #optimizer='adam',
    optimizer='adam', 
    # metrics=tf.metrics.BinaryAccuracy(threshold=0.0)
    #metrics=tf.metrics.CategoricalAccuracy()
    metrics=['accuracy']
)

In [43]:
model.fit(combined_dataset, epochs = 15, validation_data =  combined_dataset_val)

Epoch 1/15
208/208 [==============================] - 35s 168ms/step - loss: 2.3391 - accuracy: 0.2050 - val_loss: 4.5837 - val_accuracy: 0.0567
Epoch 2/15
208/208 [==============================] - 34s 166ms/step - loss: 2.3270 - accuracy: 0.2105 - val_loss: 4.6934 - val_accuracy: 0.0585
Epoch 3/15
208/208 [==============================] - 34s 166ms/step - loss: 2.3063 - accuracy: 0.2162 - val_loss: 4.7733 - val_accuracy: 0.0542
Epoch 4/15
208/208 [==============================] - 35s 166ms/step - loss: 2.2920 - accuracy: 0.2165 - val_loss: 4.8131 - val_accuracy: 0.0591
Epoch 5/15
208/208 [==============================] - 36s 171ms/step - loss: 2.2751 - accuracy: 0.2228 - val_loss: 4.8519 - val_accuracy: 0.0615
Epoch 6/15
208/208 [==============================] - 34s 165ms/step - loss: 2.2570 - accuracy: 0.2335 - val_loss: 4.9383 - val_accuracy: 0.0579
Epoch 7/15
208/208 [==============================] - 34s 164ms/step - loss: 2.2477 - accuracy: 0.2337 - val_loss: 4.9658 - val_ac